# Inspect Various Attributes

In [2]:
import sqlite3

In [2]:
import scripts.buildSqlite

10231 entries added to table SStuBs
63923 entries added to table SStuBs Large
25539 entries added to table Bugs
153652 entries added to table Bugs Large
Created indices


In [3]:
conn = sqlite3.connect('../database/sstubs.db')
cursor = conn.cursor()

### Terminologies

DB Col | Dataset Col | Def
--- | --- | --- |
parent | fixCommitParentSHA1 | SHA1 of the parent commit
child | fixCommitSHA1 | SHA1 of the current commit
type | bugType | One of the given 16 bug types
project | projectName | The project's name in format '{repo_owner}.{repo_name}'
file | bugFilePath | Relative path to the buggy file
line | bugLineNum | Line number of the buggy AST node
before | sourceBeforeFix | The fixed AST node in textual format
after | sourceAfterFix | The buggy AST node in textual format

### 1. Unique SHA1

#### 1.1. Bugs

In [4]:
query = 'SELECT count(DISTINCT child), count() FROM bugs_large'
for unq, tot in cursor.execute(query):
    print(f'There are total {unq:,} unique fixCommitSHA1 among total {tot:,} entries')
    print(f'That is each commit fixes nearly {round(tot / unq, 2)} buggy lines')

There are total 66,261 unique fixCommitSHA1 among total 153,652 entries
That is each commit fixes nearly 2.32 buggy lines


#### 1.2. SStuBs

In [5]:
query = 'SELECT count(DISTINCT child), count() FROM sstubs_large'
for unq, tot in cursor.execute(query):
    print(f'There are total {unq:,} unique fixCommitSHA1 among total {tot:,} entries')
    print(f'That is each commit contains nearly {round(tot / unq, 2)} lines of stupid bugs')

There are total 24,486 unique fixCommitSHA1 among total 63,923 entries
That is each commit contains nearly 2.61 lines of stupid bugs


### 2. Chance of introducing new bug / skipping existing bug

#### 2.1. At commit level
If the same `fixCommitSHA1` is found as the `fixCommitParentSHA1` in another commit,
then the former commit needed a subsequent fix.

##### 2.1.1. Bugs

In [6]:
query = '''SELECT count(DISTINCT child) FROM bugs_large WHERE child IN (
    SELECT parent FROM bugs_large
)'''

for row, in cursor.execute(query):
    print(f'{row:,} fixe commits required another fix')

3,357 fixe commits required another fix


##### 2.1.2. SStubs

In [7]:
query = '''SELECT count(DISTINCT child) FROM sstubs_large WHERE child IN (
    SELECT parent FROM sstubs_large
)'''

for row, in cursor.execute(query):
    print(f'{row:,} fixe commits required another fix')

582 fixe commits required another fix


#### 2.2. At line level
If the same (line, file, project) sequence contains a `fixCommitSHA1`
that was `fixCommitParentSHA1` in a previous entry, that means that line required a subsequent fix

##### 2.2.1. Bugs

In [8]:
query = '''
SELECT 1 as dummy_col
FROM bugs_large
WHERE (child, project, file, line) IN (
    SELECT parent, project, file, line FROM bugs_large
)
GROUP BY child, project, file, line'''
print(f'{len(list(cursor.execute(query))):,} line-fixes required another fix')

846 line-fixes required another fix


##### 2.2.2. SStubs

In [9]:
query = '''SELECT 1 as dummy_col FROM sstubs_large WHERE (child, project, file, line) IN (
    SELECT parent, project, file, line FROM sstubs_large
) GROUP BY child, project, file, line'''
print(f'{len(list(cursor.execute(query))):,} line-fixes required another fix')

315 line-fixes required another fix


### 3. Fix-commit tree

In [4]:
from collections import deque

#### 3.1. Get necessary data

##### 3.1.1 Get parent-child relations

Imagine a forest of the following structure
```
    a        e      g
   / \      /      /
  b   c    f      h
     /           /
    d           i
```

Now, to find a path of length 3, we must start at either `a` or `g`.
However, `tree.keys()` will have every node that has a child (e.g., `c` or `h`).
Thus, we also defined a list `treeRoots` that contains nodes without any parents.

In [5]:
tree = {}
treeRoots = []

def init_tree(db: str):
    global tree, treeRoots

    print(f'For database: {db}')

    tree = {}
    query = f'SELECT parent, child FROM {db}'
    children = list()
    for parent, child in cursor.execute(query):
        if parent not in tree:
            tree[parent] = set()
        tree[parent].add(child)
        children.append(child)

    print(f'{len(tree):,} unique parentSHA1')
    uql_child_count = len(set(children))
    print(f'{ uql_child_count :,} unique childSHA1 from {len(children):,} entry')
    tot_sha1 = len(set(tree.keys()).union(children))
    print(f'{ tot_sha1 :,} uniqueSHA1 combining children and parents')
    print(f'{tot_sha1 - uql_child_count :,} tree roots')

    query = f'''
    SELECT DISTINCT parent FROM {db} WHERE parent NOT IN (
        SELECT child FROM {db}
    )
    '''
    treeRoots = [p for p, in list(cursor.execute(query))]
    print(f'{len(treeRoots):,} tree-roots')

init_tree('sstubs_large')

For database: sstubs_large
23,784 unique parentSHA1
24,486 unique childSHA1 from 63,923 entry
47,688 uniqueSHA1 combining children and parents
23,202 tree roots
23,202 tree-roots


##### 3.1.2. Define utility function

In [6]:
def bfs(start, return_path=False, return_refix=False):
    visited_roots = set()
    distance = {start: 0}
    queue = deque()
    queue.append(start)
    # mark `start` as visited
    visited_roots.add(start)

    max_dist = 0
    distant_child = None
    # `parent_map` is used to trace back from leaf to root
    parent_map = {}
    while queue:
        p = queue.popleft()
        if p not in tree:
            continue
        for c in tree[p]:
            if c not in visited_roots:
                parent_map[c] = p
                visited_roots.add(c)
                distance[c] = distance[p] + 1
                queue.append(c)

                if distance[c] > max_dist:
                    max_dist = distance[c]
                    distant_child = c

    ret = [distant_child, max_dist]

    if return_path:
        fix_path = [distant_child]
        node = distant_child
        while node in parent_map:
            node = parent_map[node]
            fix_path.append(node)
        fix_path.reverse()
        ret.append(fix_path)

    if return_refix:
        # distance of 1 does not make a node part of a refix path.
        # first identify the nodes with distance > 1
        # then include their parents as well.
        non_parent_nodes_of_refix_paths = [node for node in distance.keys() if distance[node] > 1]
        nodes_of_refix_paths = set(non_parent_nodes_of_refix_paths)\
            .union([parent_map[c] for c in non_parent_nodes_of_refix_paths])
        ret.append(nodes_of_refix_paths)

        leaves_of_refix_paths = [
            node for node in distance.keys()
                # if `node` is not a key of `tree` then it does not have any child
                # and thus a leaf
                if distance[node] > 1 and node not in tree
        ]
        num_refix_paths = len(leaves_of_refix_paths)
        ret.append(num_refix_paths)
        ret.append(visited_roots)
    ret.append(distance)


    return ret

#### 4.2. Finding a commit sequence of length 5

In [7]:
init_tree('bugs_large')
path = []
count = 0
commitCount = 0
for root in treeRoots:
    longestPathEnd, longestDistance, *_ = bfs(root)
    if longestDistance >= 5:
        _, _, path, *_ = bfs(root, True)
        print('Path:')
        print(*path, sep=', ')
        count += 1
        commitCount += len(path)
print(f'There are total {count} path with depth >= 5 with {commitCount} commits')

For database: bugs_large
63,994 unique parentSHA1
66,261 unique childSHA1 from 153,652 entry
126,898 uniqueSHA1 combining children and parents
60,637 tree roots
60,637 tree-roots
Path:
063bc8616e8322dca47ae4b9d4860b864a61f215, 0b31e2f4558706b0831744485a80958c93524a44, 4eae69e20692a697a12a705155e972ddf448ca48, 9323424d263a1e573ab7edbfc69d67d8782ce36a, 6f74927366d17a4773006a094a7f0bc29c4b674b, c32ad40f85061b84724dd9b5b8479eebea8675a0
Path:
154a32588bfb56ed6d48e16fe64304d38ecf59f7, e1f0ce0162ca5f44a89e9f0f0cf9409c00a04a9e, 4731edc4e961df6038f18ead56be3d79eb92b7e4, 70e0f49912c83b35db962467cfab565e9c81926a, c76fee435972cbc2100bdd6e14dd86d99f28250a, c4481249285a509c7e63530cf3c55f79a6cb40f0, af0fd5a0a75424622e5dd770e7d785fd2b53e792, 13b12c5a65bace78fe91ec92e82e4e0b080a8bc1, de4eccb401ed0af0f7286aef660ef9644dabe471, 0e3f8db4f85fe7cf63843c52a0c06a6bdcbf109d, c4e45597c4f9bb3d9550ba5ec72fb67a7655532e
Path:
1c009a8c4c3c07923d8f23231496d77024010bdb, b20c6c74cc4cd9e0f674399e132a1cd42c0ad44b, e31cd00

#### 4.3. Files and corresponding changes of the found fix-path

In [8]:
parent_child = []
placeholder = []
for i in range(1, len(path)):
    query = f'''
    SELECT * FROM (
        SELECT file, group_concat(before) as before, group_concat(after) as after
        FROM bugs
        WHERE (parent=? AND child=?)
        GROUP BY parent, child, file, line
    )'''

    print(path[i-1], path[i], sep='\n', end='\n\n')
    for file, before, after in cursor.execute(query, (path[i-1], path[i])):
        print(file, before, after, sep='\n', end='\n\n')
    print('=========================================')


fda7b8158255293704b5867442bb42655e26286d
3a1537cee07a1641d703279969227bcff02d6447

3a1537cee07a1641d703279969227bcff02d6447
4c5e9439d8de7b2762f3f81010783be26b3d58ac

4c5e9439d8de7b2762f3f81010783be26b3d58ac
859f7d5e82a6816febd650a919bd98cea6b2d6da

859f7d5e82a6816febd650a919bd98cea6b2d6da
00054685fe488b213300d32bd4caca202ddab018

00054685fe488b213300d32bd4caca202ddab018
dc1b7cd4a99ec2c6f53024f9ca98201524ceaffc

dc1b7cd4a99ec2c6f53024f9ca98201524ceaffc
dc6a1dbd21d864d24ea11048379ed88bfce492ec

dc6a1dbd21d864d24ea11048379ed88bfce492ec
912a3c4a93382db8ee5380007dc289bee2f65656

912a3c4a93382db8ee5380007dc289bee2f65656
ff957c1249b28aa2e205aa3062deabb1da4f6b04

ff957c1249b28aa2e205aa3062deabb1da4f6b04
3d8758a68a57100888eddf3fc00b8b4f54d65833



#### 4.4. Number of refix

In [9]:
init_tree('sstubs_large')
total_refix = 0
visited = set()
for k in treeRoots:
    if k not in visited:
        _, _, new_refix_nodes, num_new_refix, visited_now, _ = bfs(k, return_refix=True)
        total_refix += num_new_refix
        visited = visited.union(visited_now)
print(f'There are total {total_refix} paths in sstubs_large that required at least one refix (have at least 2 child commits)')
print(f'There are {len(visited):,} commits in these paths')

For database: sstubs_large
23,784 unique parentSHA1
24,486 unique childSHA1 from 63,923 entry
47,688 uniqueSHA1 combining children and parents
23,202 tree roots
23,202 tree-roots
There are total 528 paths in sstubs_large that required at least one refix (have at least 2 child commits)
There are 47,688 commits in these paths


#### 4.5. Refix database

In [16]:
import warnings
import pandas as pd

In [17]:
databases = ['sstubs_large', 'bugs_large']
for database in databases:
    init_tree(database)
    distance = {}
    refix_path_commits = set()
    num_refix_paths = 0
    for p in treeRoots:
        if p in distance:
            warnings.warn(f'A root {p} is already visited')
            continue

        _, _, new_refix_nodes, new_refix_count, _, new_distance = bfs(p, return_refix=True)
        for node in new_refix_nodes:
            if node in refix_path_commits:
                warnings.warn(f'Node {node} is already present in a refix path')
                continue
            refix_path_commits.add(node)
        num_refix_paths += new_refix_count

        for k, v in new_distance.items():
            if k in distance:
                warnings.warn(f'Key: {k}, old dist: {distance[k]}, new dist: {new_distance[k]}')
                continue
            distance[k] = new_distance[k]

    print(f'{num_refix_paths:,} refix paths')
    print(f'{len(refix_path_commits):,} nodes involving refix in {database}\n')

    data = [
        [key, value, key in refix_path_commits] for key, value in distance.items()
    ]
    pd.DataFrame(
        data=data,
        columns=['commit', 'distance', 'refix'],
    ).to_csv(f'../database/{database}_distances.csv', index=False)

For database: sstubs_large
23,784 unique parentSHA1
24,486 unique childSHA1 from 63,923 entry
47,688 uniqueSHA1 combining children and parents
23,202 tree roots
23,202 tree-roots
528 refix paths
1,110 nodes involving refix in sstubs_large

For database: bugs_large
63,994 unique parentSHA1
66,261 unique childSHA1 from 153,652 entry
126,898 uniqueSHA1 combining children and parents
60,637 tree roots
60,637 tree-roots
2,983 refix paths
6,340 nodes involving refix in bugs_large



### 5. Number of Owner and Projects

In [18]:
query = '''SELECT count(DISTINCT project) FROM bugs'''
for row in cursor.execute(query):
    print(f'There are {row[0]} projects / repo')

There are 96 projects / repo


In [19]:
query = '''SELECT count(DISTINCT substr(project, 0, instr(project, '.'))) FROM bugs_large'''
for row in cursor.execute(query):
    print(f'There are {row[0]} repo owners')

There are 558 repo owners


In [20]:
query = '''
SELECT count()
FROM (
    SELECT *
    FROM (
        SELECT
           substr(project, 0, instr(project, '.')) as owner,
           substr(project, instr(project, '.') + 1) as repo
        FROM bugs_large) AS owners
    GROUP BY owner
    HAVING count(DISTINCT repo) > 1
)'''
for row in cursor.execute(query):
    print(f'There are {row[0]} repo owners having multiple projects / repo')

There are 57 repo owners having multiple projects / repo


In [21]:
query = '''
SELECT count()
FROM (
    SELECT DISTINCT owner
    FROM (
        SELECT
           substr(project, 0, instr(project, '.')) as owner,
           substr(project, instr(project, '.') + 1) as repo
        FROM bugs_large) AS owners
    WHERE owner = repo
)'''
for row in cursor.execute(query):
    print(f'There are {row[0]} repo having the same name as owner')

There are 65 repo having the same name as owner


### 6. Sources That Required Sub-Sequent Changes

In [22]:
import pandas as pd
pd.set_option('display.max_rows', 0)

In [23]:
query = '''
SELECT a.before as first, a.after as second, b.after as third
FROM sstubs AS a
    INNER JOIN sstubs as b on a.after = b.before
WHERE 5 < length(b.before) AND length(b.before) < 12
'''
pd.DataFrame(cursor.execute(query), columns=['First', 'Second', 'Third'])

,First,Second,Third
0,IOException e,Exception e,Throwable e
1,NumberFormatException nfe,Exception e,Throwable e
2,IllegalArgumentException e,Exception e,Throwable e
3,IllegalArgumentException e,Exception e,Throwable e
4,ReflectiveOperationException e,Exception e,Throwable e
5,ReflectiveOperationException e,Exception e,Throwable e
6,IOException e,Exception e,Throwable e
7,IOException e,Exception e,Throwable e
8,IOException e,Exception e,Throwable e
...,...,...,...


### 7. Length of SHA1 to Uniquely identify a commit

In [24]:
for table in ['bugs', 'bugs_large', 'sstubs', 'sstubs_large']:
    # noinspection SqlResolve
    query = f'''
    SELECT count()
    FROM (
         SELECT parent FROM {table} UNION SELECT child FROM {table}
    )
    '''
    numUnqCommits = next(cursor.execute(query))
    for i in range(40):
        # noinspection SqlResolve
        query = f'''
        SELECT count(DISTINCT substr(parent, 0, {i}))
        FROM (
             SELECT parent FROM {table} UNION SELECT child FROM {table}
        )
        '''
        numIdentifiableCommits = next(cursor.execute(query))
        if numUnqCommits == numIdentifiableCommits:
            print(f'For {table} {i} chars are enough to identify a commit SHA1')
            break

For bugs 9 chars are enough to identify a commit SHA1
For bugs_large 10 chars are enough to identify a commit SHA1
For sstubs 8 chars are enough to identify a commit SHA1
For sstubs_large 9 chars are enough to identify a commit SHA1
